### ДЗ по NLP №2


1. выделить частотные слова, сделать на их основании словарик хороших из них, увеличить его с помощью эмбеддингов, искать н-граммы (это я и буду реализовывать)
2. что-нибудь связанное с шаблонами: составить шаблоны, в которых могут располагаться именованные сущности и по ним поискать. Можно сделать это как продолжение первого пункта: посмотреть на частотные слова и их контексты, из этого уже составлять шаблоны
3. можно взять какой-нибудь готовый список названий товаров и искать совпадения. это не поможет нам выделить слова типа "шампунь", но мы можем посмотреть на слова, которые часто встречаются рядом с названиями косметики, и там частотными скорее всего будут как раз слова типа "тушь", "шампунь" и тд, то есть то что нам нужно.

In [1]:
import gzip
import re
import pandas as pd
import json

In [44]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

In [45]:
data = parse('reviews_Beauty_5.json.gz')

Вынимаем тексты

In [46]:
texts = []
for i in data:
  texts.append(i['reviewText'])
  if len(texts) > 100000:
    break

In [47]:
texts = list(set(texts))

In [48]:
len(texts)

99971

In [8]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [49]:
def normalize(text):
    text = text.lower()
    text = re.sub('[.|,|"|`]', '', text)
    tokens = [w for w in nltk.word_tokenize(text) if w not in '1234567890,.!?-();:""«»—_–#' and '.' not in w]
    lemmas = [lemmatizer.lemmatize(t) for t in tokens]
    return ' '.join(lemmas)

Делаем лемматизированные тексты

In [50]:
n_texts = []
for text in texts:
  n_texts.append(normalize(text))

In [12]:
!pip install python-rake

In [19]:
import RAKE
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
rake = RAKE.Rake(stop)

Достаем ключевые слова, чтобы на их основании выделить самостоятельно те из них, которые нам кажутся хорошими

In [52]:
kws = []
for i in n_texts:
    kw = rake.run(i, maxWords=3, minFrequency=1)
    for k in kw:
        kws.append(k[0])

In [53]:
from collections import Counter
Counter(kws).most_common(100)

[('n', 26574),
 ('use', 25318),
 ('product', 22475),
 ('love', 15824),
 ('hair', 14573),
 ('like', 13650),
 ('used', 13327),
 ('skin', 12162),
 ('using', 11449),
 ('get', 11206),
 ('wa', 10786),
 ('make', 10511),
 ('one', 9643),
 ('doe n', 8333),
 ('work', 8091),
 ('well', 8017),
 ('face', 7807),
 ('day', 7781),
 ('time', 7626),
 ('think', 7393),
 ('try', 7017),
 ('bought', 6935),
 ('doe', 6926),
 ('lot', 6671),
 ('ha', 6643),
 ('much', 6475),
 ('need', 6273),
 ('color', 6155),
 ('smell', 6112),
 ('give', 6089),
 ('great', 6027),
 ('good', 5949),
 ('tried', 5947),
 ('put', 5940),
 ('go', 5807),
 ('dry', 5663),
 ('price', 5572),
 ('know', 5528),
 ('buy', 5460),
 ('want', 5459),
 ('would', 5380),
 ('see', 5350),
 ('find', 5284),
 ('say', 5258),
 ('found', 4954),
 ('year', 4876),
 ('ca n', 4740),
 ('got', 4709),
 ('way', 4559),
 ('week', 4513),
 ('keep', 4279),
 ('last', 4134),
 ('apply', 4083),
 ('sure', 3855),
 ('easy', 3801),
 ('feel', 3775),
 ('take', 3773),
 ('brush', 3668),
 ('stuff

Составляем свой список хороших слов

In [23]:
words = ['product', 'hair', 'skin', 'face', 'brush', 'bottle', 'scent', 'wash', 'conditioner', 'lotion']

In [24]:
!wget http://vectors.nlpl.eu/repository/20/5.zip

--2021-12-04 17:28:06--  http://vectors.nlpl.eu/repository/20/5.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 574653290 (548M) [application/zip]
Saving to: ‘5.zip’

5.zip               100%[===================>] 548.03M  22.6MB/s    in 25s     

2021-12-04 17:28:32 (22.0 MB/s) - ‘5.zip’ saved [574653290/574653290]



In [25]:
!unzip '5.zip'

Archive:  5.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


Добавляем слова, похожие на хорошие.

In [26]:
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format('model.bin', binary=True)
model.save_word2vec_format('model.txt', binary=False)
def find_similar(word, model=model):
    if word in model:
        return model.most_similar(word, topn=1)[0][0]

In [54]:
new = []
for word in words:
    new_word = find_similar(word)
    if new_word not in words and new_word != None:
        new.append(new_word)
words.extend(new)

Вот они:

In [55]:
words

['product',
 'hair',
 'skin',
 'face',
 'brush',
 'bottle',
 'scent',
 'wash',
 'conditioner',
 'lotion',
 'consumer',
 'beard',
 'mucous',
 'overcome',
 'brushing',
 'bottled',
 'odor',
 'rinse',
 'dehumidifier',
 'shampoo',
 'marketer',
 'moustache',
 'mucosa',
 'spite',
 'carbonated',
 'odour',
 'soak',
 'humidifier']

Вытаскиваем биграммы

In [56]:
def context(text):
    text = text.split()
    bigrams = []
    for w in text:
        if w in words:
            n = text.index(w)
            try:
              bigrams.append(' '.join([text[n-1], text[n]]))
              bigrams.append(' '.join([text[n], text[n+1]]))
            except IndexError:
                continue
    return bigrams

In [57]:
keygrams = []
for i in n_texts:
  keygrams.extend(context(i))

In [59]:
Counter(keygrams).most_common(50)

[('this product', 30383),
 ('my hair', 27135),
 ('my skin', 18378),
 ('my face', 11268),
 ('hair and', 7953),
 ('product i', 7036),
 ('skin and', 6163),
 ('the product', 6136),
 ('hair i', 5541),
 ('your hair', 4747),
 ('your skin', 4648),
 ('product is', 4570),
 ('product and', 4385),
 ('product for', 4373),
 ('the skin', 4217),
 ('hair is', 4055),
 ('sensitive skin', 3716),
 ('skin i', 3711),
 ('the scent', 3316),
 ('the bottle', 3293),
 ('dry skin', 2879),
 ('product that', 2799),
 ('hair that', 2737),
 ('skin is', 2541),
 ('your face', 2517),
 ('curly hair', 2429),
 ('product it', 2410),
 ('this brush', 2293),
 ('product to', 2245),
 ('hair product', 2238),
 ('the hair', 2221),
 ('this lotion', 2191),
 ('this shampoo', 2170),
 ('face and', 2147),
 ('hair it', 2118),
 ('body wash', 2081),
 ('face wash', 2081),
 ('the brush', 2080),
 ('shampoo and', 2047),
 ('great product', 2044),
 ('hair dryer', 1939),
 ('product but', 1847),
 ('fine hair', 1747),
 ('scent is', 1742),
 ('hair but',

Все, биграммы вытащили, можно переходить к метрикам.

In [32]:
from nltk.collocations import *

In [33]:
m = nltk.collocations.BigramAssocMeasures()

In [60]:
list_bigrams = []
for i in keygrams:
  bi = i.split(' ')
  list_bigrams.append(bi)

In [64]:
finder = BigramCollocationFinder.from_documents(list_bigrams)

In [65]:
likelihood = finder.score_ngrams(m.likelihood_ratio)
likelihood[:20]

[(('this', 'product'), 83123.24919164405),
 (('hair', 'hair'), 45141.90119643502),
 (('product', 'hair'), 40820.54129777351),
 (('my', 'hair'), 38641.58719553852),
 (('product', 'product'), 36793.742519346706),
 (('skin', 'hair'), 30173.782715461053),
 (('hair', 'skin'), 29483.885885844677),
 (('product', 'skin'), 27571.8398757189),
 (('hair', 'product'), 26018.20091885026),
 (('my', 'skin'), 23412.302231723068),
 (('skin', 'product'), 23269.27823813095),
 (('my', 'face'), 22496.377324083358),
 (('skin', 'skin'), 20515.476833806704),
 (('sensitive', 'skin'), 17233.03908583506),
 (('my', 'product'), 14610.462206377244),
 (('body', 'wash'), 12981.425690976583),
 (('hair', 'my'), 11746.745382553696),
 (('face', 'hair'), 11607.026889232424),
 (('hair', 'face'), 11376.317886793542),
 (('product', 'my'), 11277.878118990973)]

In [66]:
pmi = finder.score_ngrams(m.pmi)
pmi[:20]

[(('odour', 'ortaste'), 16.001322666237),
 (('mucous', 'eewww'), 15.711816049042017),
 (('mucous', 'membrane'), 15.711816049042016),
 (('tankless', 'humidifier'), 15.363892745621708),
 (('smart', 'marketer'), 15.264357072070794),
 (("'ve", 'bottled'), 14.470807949538221),
 (('bottled', 'witch'), 14.470807949538221),
 (('filtered', 'bottled'), 14.470807949538221),
 (('togetherand', 'bottled'), 14.470807949538221),
 (('captivating', 'odour'), 14.416360165515844),
 (('pungent', 'odour'), 14.001322666237002),
 (('gooey', 'mucous'), 13.904461126984412),
 (('room', 'humidifier'), 13.778930244900554),
 (('carbonated', 'drink'), 13.711816049042016),
 (('sorry', 'mucous'), 13.12685354832086),
 (('dehumidifier', 'highly'), 13.083784826428975),
 (('mist', 'humidifier'), 12.778930244900554),
 (("'acne", 'beard'), 12.703642117596317),
 (('beard', 'stood'), 12.703642117596317),
 (('trimmed', 'beard'), 12.703642117596317)]

In [67]:
dice = finder.score_ngrams(m.dice)
dice[:20]

[(('this', 'product'), 0.3311949856928737),
 (('mucous', 'membrane'), 0.3076923076923077),
 (('my', 'hair'), 0.25284432393145667),
 (('my', 'skin'), 0.21937724936883382),
 (('my', 'face'), 0.21698231289897074),
 (('body', 'wash'), 0.18398019626911855),
 (('mucous', 'eewww'), 0.16666666666666666),
 (('smart', 'marketer'), 0.15384615384615385),
 (('gooey', 'mucous'), 0.13793103448275862),
 (('tankless', 'humidifier'), 0.13333333333333333),
 (('the', 'bottle'), 0.1299475158833511),
 (('the', 'scent'), 0.1288192218790669),
 (('consumer', 'report'), 0.11946902654867257),
 (('room', 'humidifier'), 0.11764705882352941),
 (('odour', 'ortaste'), 0.10526315789473684),
 (('rinse', 'off'), 0.10293571163136381),
 (('soak', 'into'), 0.1013745704467354),
 (('bottle', 'of'), 0.09875074360499703),
 (('captivating', 'odour'), 0.09523809523809523),
 (('your', 'face'), 0.09262700793052055)]

Честно говоря, все работает ужасно...
Но давайте считать что лучше всего работает dice, кажется, это похоже на правду. Из лучших по его мнению и будем брать биграммы, которые должны лучше всего соответствовать пяти словам, которые я выбрала

In [69]:
set_bi = list(set(keygrams))

In [70]:
best = ['product', 'hair', 'skin', 'face', 'shampoo']

In [75]:
colls = []
for j in range(len(best)):
  colls.append([])
  for i in dice[:100]:
    if best[j] == i[0][0] or best[j] == i[0][1]:
      colls[j].append(i[0])

In [77]:
for i in range(len(colls)):
  print('<<<'+best[i]+'>>>')
  for j in colls[i]:
    print(' '.join(j))

<<<product>>>
this product
product i
the product
product for
product is
product and
product that
<<<hair>>>
my hair
hair and
hair i
your hair
hair is
<<<skin>>>
my skin
skin and
your skin
sensitive skin
the skin
skin i
dry skin
skin is
<<<face>>>
my face
your face
face wash
face and
face i
<<<shampoo>>>
shampoo and
this shampoo


Вот и они. Как мы можем видеть, все работает очень плохо и достает какой-то шлак. Надо было выбирать другой алгоритм)))))